<a href="https://colab.research.google.com/github/Rishabhjuneja/NYC-Taxi/blob/main/exploartion_pyspark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# innstall java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# install spark (change the version number if needed)
!wget -q https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz

# unzip the spark file to the current folder
!tar xf spark-3.0.0-bin-hadoop3.2.tgz

!pip install -q findspark

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop3.2"

In [3]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [10]:
from pyspark.sql import functions as F

In [5]:
file_loc='./green_tripdata_2016-02.csv'
df = spark.read.csv(file_loc, inferSchema=True, header =True)

In [6]:
df.show(5)

+--------+--------------------+---------------------+------------------+----------+------------------+------------------+------------------+------------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+----------+
|VendorID|lpep_pickup_datetime|Lpep_dropoff_datetime|Store_and_fwd_flag|RateCodeID|  Pickup_longitude|   Pickup_latitude| Dropoff_longitude|  Dropoff_latitude|Passenger_count|Trip_distance|Fare_amount|Extra|MTA_tax|Tip_amount|Tolls_amount|Ehail_fee|improvement_surcharge|Total_amount|Payment_type|Trip_type |
+--------+--------------------+---------------------+------------------+----------+------------------+------------------+------------------+------------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+----------+
|       2| 2016-02-01 00:00:01|  2016-02-01 00:10:06|                 N| 

In [7]:
# check number of rows & Coloums 

print((df.count(), len(df.columns)))

(452861, 21)


In [12]:
# Check any missing values 

from pyspark.sql.functions import lit

rows = df.count()
summary = df.describe().filter(F.col("summary") == "count")
summary.select(*((F.lit(rows)-F.col(c)).alias(c) for c in df.columns)).show()

+--------+--------------------+---------------------+------------------+----------+----------------+---------------+-----------------+----------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+----------+
|VendorID|lpep_pickup_datetime|Lpep_dropoff_datetime|Store_and_fwd_flag|RateCodeID|Pickup_longitude|Pickup_latitude|Dropoff_longitude|Dropoff_latitude|Passenger_count|Trip_distance|Fare_amount|Extra|MTA_tax|Tip_amount|Tolls_amount|Ehail_fee|improvement_surcharge|Total_amount|Payment_type|Trip_type |
+--------+--------------------+---------------------+------------------+----------+----------------+---------------+-----------------+----------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+----------+
|     0.0|                 0.0|                  0.0|               0.0|       0.0|             0

In [13]:
# Descriptive Statistics

df.describe().show()

# Look into ratecodeid , trip distance , fare amount, extra , mat tax should be .5 , trip amount , 

+-------+-------------------+--------------------+---------------------+------------------+------------------+-------------------+------------------+-------------------+------------------+------------------+------------------+------------------+------------------+-------------------+------------------+-------------------+---------+---------------------+------------------+------------------+-------------------+
|summary|           VendorID|lpep_pickup_datetime|Lpep_dropoff_datetime|Store_and_fwd_flag|        RateCodeID|   Pickup_longitude|   Pickup_latitude|  Dropoff_longitude|  Dropoff_latitude|   Passenger_count|     Trip_distance|       Fare_amount|             Extra|            MTA_tax|        Tip_amount|       Tolls_amount|Ehail_fee|improvement_surcharge|      Total_amount|      Payment_type|         Trip_type |
+-------+-------------------+--------------------+---------------------+------------------+------------------+-------------------+------------------+-------------------+---

In [15]:
pandasDF = df.toPandas()
print(pandasDF)

        VendorID lpep_pickup_datetime  ... Payment_type Trip_type 
0              2  2016-02-01 00:00:01  ...            2        1.0
1              2  2016-02-01 00:01:33  ...            2        1.0
2              2  2016-02-01 00:03:46  ...            1        1.0
3              2  2016-02-01 00:00:05  ...            2        1.0
4              2  2016-02-01 00:06:20  ...            2        1.0
...          ...                  ...  ...          ...        ...
452856         2  2016-02-10 11:21:59  ...            2        2.0
452857         2  2016-02-10 11:07:37  ...            1        1.0
452858         2  2016-02-10 11:41:05  ...            2        1.0
452859         2  2016-02-10 11:39:43  ...            2        1.0
452860         2  2016-02-10 11:39:32  ...            2        1.0

[452861 rows x 21 columns]


In [ ]:
df.quantile(q=0.5, axis=0, numeric_only=False, interpolation='linear')